In [1]:
%%shell
rm -rf bob.paper.ijcb2024_agnostic_features_mad/
git clone https://gitlab.idiap.ch/bob/bob.paper.ijcb2024_agnostic_features_mad.git

Cloning into 'bob.paper.ijcb2024_agnostic_features_mad'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 180 (delta 25), reused 0 (delta 0), pack-reused 98 (from 1)
Receiving objects: 100% (180/180), 307.95 KiB | 2.25 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
# connect to drive to have access to shared FFHQ dataset

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Download FFHQ-Morphs dataset and extract data -> /content/datasets/ffhq-morphs/FFHQ-Morphs/
#%%shell
#wget -O ffhq-morphs.zip https://zenodo.org/api/records/14752744/files-archive
#mkdir -p /content/datasets/ffhq-morphs
#unzip -q ffhq-morphs.zip -d /content/datasets/ffhq-morphs
#find /content/datasets/ffhq-morphs/ -name "*.tar.gz" -exec tar -xvzf {} -C /content/datasets/ffhq-morphs/ \;

In [4]:
# Update dataset.csv file
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/datasets.csv')


# Define the old and new base paths
path_mappings = {
    '/idiap/user/lcolbois/datasets/morphs/ffhq/': '/content/drive/MyDrive/mad/datasets/ffhq-morphs/',
    '/idiap/resource/database/Flickr-Faces-HQ/': '/content/drive/MyDrive/mad/datasets/ffhq-dataset/'
}

# Update the 'path' column based on the defined mappings
for old_base_path, new_base_path in path_mappings.items():
    mask = df['path'].str.startswith(old_base_path)
    df.loc[mask, 'path'] = df.loc[mask, 'path'].str.replace(old_base_path, new_base_path, regex=False)

# Identify the rows where 'dataset' is 'ffhq'
mask = df['dataset'] == 'ffhq'

# Filter the DataFrame to keep only the rows where 'dataset' is 'ffhq'
df_filtered = df[mask]


# Save the updated DataFrame back to a CSV file
df_filtered.to_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/datasets.csv', index=False)

In [5]:
# Update gathered_sets.csv file
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/gathered_sets.csv')

# Identify the rows where 'dataset' is 'ffhq'
mask = df['src_dataset'] == 'ffhq'

# Filter the DataFrame to keep only the rows where 'dataset' is 'ffhq'
df_filtered = df[mask]

# Save the updated DataFrame back to a CSV file
df_filtered.to_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/gathered_sets.csv', index=False)

In [6]:
# Remove mention of other datasets from experiments file
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/experiments/detection.csv')

# Filter the DataFrame to keep only the rows where 'dataset' is 'ffhq'
df_filtered = df[df['train_set'].str.startswith('ffhq') & df['eval_set'].str.startswith('ffhq')]

# Save the updated DataFrame back to a CSV file
df_filtered.to_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/experiments/detection.csv', index=False)



# Load the CSV file into a DataFrame
df = pd.read_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/experiments/one_class.csv')

# Identify the rows where 'train_set' is 'ffhq'
df_filtered = df[df['train_set'].str.startswith('ffhq') & df['eval_set'].str.startswith('ffhq')]

# Save the updated DataFrame back to a CSV file
df_filtered.to_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/experiments/one_class.csv', index=False)



# Load the CSV file into a DataFrame
df = pd.read_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/experiments/tsne.csv')

# Identify the rows where 'train_set' is 'ffhq'
mask = df['train_set'].str.startswith('ffhq')

# Filter the DataFrame to keep only the rows where 'dataset' is 'ffhq'
df_filtered = df[mask]

# Save the updated DataFrame back to a CSV file
df_filtered.to_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/experiments/tsne.csv', index=False)

In [7]:
# FIX: git clone in external.smk

# Define the path to your external.smk file
file_path = '/content/bob.paper.ijcb2024_agnostic_features_mad/rules/external.smk'

# Define the exact line to be replaced and its replacement
original_line = '        "git clone --single-branch --branch mada git@github.com:lcolbois/SMDD-Synthetic-Face-Morphing-Attack-Detection-Development-dataset.git src/mada/contrib/mixfacenet_mad"'
replacement_line = '        "git clone https://github.com/naserdamer/SMDD-Synthetic-Face-Morphing-Attack-Detection-Development-dataset.git src/mada/contrib/mixfacenet_mad"'

# Initialize a flag to check if the line was replaced
line_replaced = False

# Open the file for reading and writing
with open(file_path, 'r+') as file:
    lines = file.readlines()
    file.seek(0)  # Move the cursor to the beginning of the file
    for line in lines:
        # Check if the line matches the original line
        if line.strip() == original_line.strip():
            file.write(replacement_line + '\n')
            line_replaced = True
        else:
            file.write(line)

    # Truncate the file to the current position to remove any leftover content
    file.truncate()

# Inform the user about the result
if line_replaced:
    print("Line successfully replaced.")
else:
    print("Original line not found. No changes made.")

Line successfully replaced.


In [8]:
# Remove not working extractors
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/extractors.csv')

# Filter the DataFrame to keep only the working extractors
df_filtered = df[~df['name'].isin(['dinov2_giant', 'dnadet_pretrain', 'resnet50_imagenet', 'clip_vit_l14'])]

# Save the updated DataFrame back to a CSV file
df_filtered.to_csv('/content/bob.paper.ijcb2024_agnostic_features_mad/protocols/extractors.csv', index=False)

In [9]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [10]:
%%shell
cd bob.paper.ijcb2024_agnostic_features_mad
eval "$(conda shell.bash hook)" # copy conda command to shell
conda remove --name mada --all -y
conda env create -f environment.yml
conda activate mada
pip install -e .

Streaming output truncated to the last 5000 lines.








pyside6-6.9.2        | 9.7 MB    | : 100% 1.0/1 [00:09<00:00,  4.85s/it]               
















pyside6-6.9.2        | 9.7 MB    | : 100% 1.0/1 [00:09<00:00,  4.85s/it]

















scikit-learn-1.7.2   | 9.2 MB    | : 100% 1.0/1 [00:09<00:00,  4.62s/it]               


libllvm21-21.1.0     | 42.3 MB   | : 100% 1.0/1 [00:12<00:00,  1.30s/it]



libopencv-4.12.0     | 31.1 MB   | : 100% 1.0/1 [00:12<00:00,  4.54s/it]
libtorch-2.7.1       | 55.7 MB   | : 100% 1.0/1 [00:24<00:00,  1.46s/it]

qt6-main-6.9.2       | 50.0 MB   | : 100% 1.0/1 [00:25<00:00,  1.43s/it]




mkl-2024.2.2         | 119.2 MB  | : 100% 1.0/1 [00:36<00:00,  5.07s/it]





pytorch-2.7.1        | 28.1 MB   | : 100% 1.0/1 [00:37<00:00,  7.94s/it]









pandas-2.3.2         | 14.4 MB   | : 100% 1.0/1 [00:38<00:00,  1.59s/it]








scipy-1.16.2         | 16.3 MB   | : 100% 1.0/1 [00:39<00:00,  1.68s/it]










ffmpeg-8.0.0         | 11.9 MB   

In [11]:
# Download SPL-MAD pre trained weights
# https://github.com/meilfang/SPL-MAD
#  https://drive.google.com/file/d/1kFLp1dWp_sBwC-l-RTVo-LRitKSYxbyv/view
!wget -O /content/casia_smdd.pth "https://drive.usercontent.google.com/download?id=1kFLp1dWp_sBwC-l-RTVo-LRitKSYxbyv&export=download&authuser=0&confirm=t&uuid=a71bfc7e-3ea3-4160-8cce-1cbb6ab3c8b2&at=AEz70l5fm0biHGmCvL3ZfsGxZ-u4%3A1743535124500"

--2025-09-20 16:18:53--  https://drive.usercontent.google.com/download?id=1kFLp1dWp_sBwC-l-RTVo-LRitKSYxbyv&export=download&authuser=0&confirm=t&uuid=a71bfc7e-3ea3-4160-8cce-1cbb6ab3c8b2&at=AEz70l5fm0biHGmCvL3ZfsGxZ-u4%3A1743535124500
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.127.132, 2a00:1450:4013:c04::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40510953 (39M) [application/octet-stream]
Saving to: ‘/content/casia_smdd.pth’

/content/casia_smdd 100%[===================>]  38.63M  98.4MB/s    in 0.4s    

2025-09-20 16:18:54 (98.4 MB/s) - ‘/content/casia_smdd.pth’ saved [40510953/40510953]



In [12]:
%%shell
source activate mada
rc set weights.spl_mad /content/casia_smdd.pth

In [13]:
%%shell
source activate mada
rc set results.temp /content/temp_results

In [14]:
%%shell
source activate mada
rc set weights.root /content/weights

In [15]:
%%shell
source activate mada
cd bob.paper.ijcb2024_agnostic_features_mad
export HYDRA_FULL_ERROR=1
snakemake -c1 all_ijcb

Assuming unrestricted shared filesystem usage.
host: 69f75a8d4f29
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                          count
-------------------------  -------
all_ijcb                         1
clone_mixfacenet_mad_repo        1
clone_spl_mad_repo               1
evaluation                       8
extraction                       6
mixfacenet_mad_evaluation        7
mixfacenet_mad_extraction       42
mixfacenet_mad_training          7
modeling                         8
preprocessing                    6
spl_evaluation                   1
total                           88

Select jobs to execute...
Failed to solve scheduling problem with ILP solver, falling back to greedy scheduler. You likely have to fix your ILP solver installation. Error message: PULP_CBC_CMD: Not Available (check permissions on cbc)
Execute 1 jobs...

[Sat Sep 20 16:19:02 202

CalledProcessError: Command 'source activate mada
cd bob.paper.ijcb2024_agnostic_features_mad
export HYDRA_FULL_ERROR=1
snakemake -c1 all_ijcb
' returned non-zero exit status 1.

In [ ]:
# from google.colab import runtime
# runtime.unassign()